<a href="https://colab.research.google.com/github/yasinelehmiani/deep-learning/blob/main/ellipse_extrct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ellipse extaction for one image

In [15]:
from google.colab.drive import mount
mount('/content/drive')

Mounted at /content/drive


In [13]:
x='/content/drive/Shareddrives/MyDrive/fetal data/Patient00168_Plane3_1_of_3_0.png'

In [16]:
x.split("/")[-1]

'Patient00168_Plane3_1_of_3_0.png'

In [ ]:
import cv2

# Load the binary mask
binary_mask = cv2.imread('/content/drive/Shareddrives/MyDrive/fetal data/Patient00168_Plane3_1_of_3_0.png', cv2.IMREAD_GRAYSCALE)

# Find contours
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Fit ellipses
ellipses = []
for contour in contours:
    if len(contour) >= 5:  # Fit ellipse requires at least 5 points
        ellipse = cv2.fitEllipse(contour)
        ellipses.append(ellipse)

# Convert Pixel Size to mm
pixel_size_mm = 0.1  # Example conversion factor
for ellipse in ellipses:
    ellipse_center_mm = (ellipse[0][0] * pixel_size_mm, ellipse[0][1] * pixel_size_mm)
    ellipse_axes_mm = (ellipse[1][0] * pixel_size_mm, ellipse[1][1] * pixel_size_mm)
    # You can now use ellipse_center_mm and ellipse_axes_mm in mm.


In [39]:
def ellipsParameers(img,name):
  name=name.split("/")[-1]
  img = cv2.imread(img)
  hh, ww = img.shape[:2]
  # convert to grayscale
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 252, 255, cv2.THRESH_BINARY)[1]
  # fit ellipse
  # note: transpose needed to convert y,x points from numpy to x,y for opencv
  points = np.column_stack(np.where(thresh.transpose() > 0))
  hull = cv2.convexHull(points)
  ((centx,centy), (width,height), angle) = cv2.fitEllipse(hull)
  # draw ellipse on input image
  result = img.copy()
  cv2.ellipse(result, (int(centx),int(centy)), (int(width/2),int(height/2)), angle, 0, 360, (0,0,255), 2)
  return [name,centx,centy,width/2,height/2,angle]



In [ ]:
elp='/content/drive/Shareddrives/MyDrive/fetal data/Patient00168_Plane3_1_of_3_0.png'

In [30]:
import pandas as pd
import numpy as np
data={"name":[ ],
      "cx":[ ] ,"cy":[ ] ,"majoraxis":[  ] ,"minoraxis":[ ] ,"angle": [ ]}
ellipse_data=pd.DataFrame(data)

In [31]:
ellipse_data.loc[1]=[1,2,3,4,5,6]

,name,cx,cy,majoraxis,minoraxis,angle
1,1,2,3,4,5,6


In [64]:
from tqdm import tqdm
from glob import glob
import pandas as pd
import numpy as np
import os
import cv2
def ellipse_all(image,mask):
  #creat a datafram in order to store the data in
  data={"name":[ ],
      "cx":[ ] ,"cy":[ ] ,"majoraxis":[  ] ,"minoraxis":[ ] ,"angle": [ ]}
  ellipse_data=pd.DataFrame(data)
  #store the dir of images and masks in a listes
  image= sorted(glob(os.path.join(image,"image","*.png")))
  mask=sorted(glob(os.path.join(mask,"mask1","*.png")))
  for id,(x,y)in tqdm(enumerate(zip(image,mask)), total=len(image)):
   # the image with the index 1232 has not a ellipse so we will pass it
    if id==1232:
      continue
    #exctract the data
    local_data= ellipsParameers(y,x)
    #store the data in the dataframe
    ellipse_data.loc[id]=local_data
  return ellipse_data











In [59]:
image="/content/drive/Shareddrives/MyDrive/fetal us data/"
masks="/content/drive/Shareddrives/MyDrive/fetal us data/"

In [60]:
data=ellipse_all(image,masks)

100%|██████████| 2564/2564 [01:53<00:00, 22.53it/s]


In [61]:
data

,name,cx,cy,majoraxis,minoraxis,angle
0,100_2HC.png,360.195801,379.335022,93.763054,109.468056,91.200073
1,101_HC.png,356.963165,400.084473,83.724518,97.330544,86.690926
2,102_2HC.png,415.178040,247.712891,194.076447,237.659531,140.925446
3,103_HC.png,353.525848,274.299896,211.415939,249.378891,141.160645
4,104_HC.png,455.181030,220.144592,160.060593,174.665710,142.880356
...,...,...,...,...,...,...
2559,Patient01789_Plane3_1_of_3.png,452.996552,278.993622,182.361801,234.298630,95.021187
2560,Patient01790_Plane3_2_of_2.png,494.001709,322.008789,285.336487,349.223389,91.993294
2561,Patient01791_Plane3_2_of_3.png,482.001099,274.000977,219.352325,259.301910,90.928337
2562,Patient01791_Plane3_3_of_3.png,497.023621,270.001526,214.343948,259.244476,90.000488


In [63]:
# prompt: Avec le DataFrame data: to csv file and exel file  and put the them in my drive

# Save the dataframe to a CSV file
data.to_csv('/content/drive/Shareddrives/MyDrive/data.csv')

# Save the dataframe to an Excel file
data.to_excel('/content/drive/Shareddrives/MyDrive/data.xlsx')
